## Rhelogy Models for Predicting Values of UV-curable Inkjet Components

In [1]:
import sklearn
import sklearn.linear_model
import time
import seaborn as sb
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
import pandas as pd
import shap
import xgboost
import pickle

from numpy import arange
from pandas import read_csv
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn_genetic import GASearchCV
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from skopt import BayesSearchCV
from sklearn_genetic.space import Integer, Categorical, Continuous
from mapie.regression import MapieRegressor
from sklearn.neural_network import MLPRegressor

## Data Wrangling

In [2]:
rocky = pd.read_excel('FINAL-trainingset.xlsx')

In [3]:
rocky.columns[0:3]

Index(['Surface Tension', 'Viscosity', 'Density'], dtype='object')

In [4]:
rocky = rocky.drop(columns=['ID','Molecule_Name','SMILES'])
#rocky = rocky.drop(columns=['Surface Tension'])
rocky = rocky.drop(columns=['Viscosity'])
rocky = rocky.drop(columns=['Density'])
print(rocky)

     Surface Tension      MW     AMW      Sv      Se      Sp      Si     Mv  \
0                NaN   78.50  11.214   4.596   7.418   4.835   7.984  0.657   
1              25.90  137.03   9.788   7.659  13.647   9.159  15.918  0.547   
2              25.26  123.00  11.182   6.132  10.764   7.398  12.502  0.557   
3              24.93   74.14   4.943   7.349  14.745   8.262  17.285  0.490   
4              23.18   92.58   6.613   7.461  13.742   8.665  16.020  0.533   
..               ...     ...     ...     ...     ...     ...     ...    ...   
265              NaN  267.83  53.566   4.687   4.905   7.841   5.272  0.937   
266            47.10   90.14   5.634   8.064  16.073   8.716  18.495  0.504   
267            44.40  207.07  11.504  13.132  17.764  14.398  19.502  0.730   
268            31.10  390.62   5.918  36.868  65.098  40.285  74.726  0.559   
269            26.70   53.07   7.581   4.548   6.985   4.767   7.914  0.650   

        Me     Mp  ...  F08_C-C_  F08_C-N_  F08_C-O

In [5]:
rocky.shape

(270, 1059)

In [6]:
#rocky1 = rocky.dropna(axis=0, subset=['Density'])
#rocky1 = rocky.dropna(axis=0, subset=['Viscosity'])
rocky1 = rocky.dropna(axis=0, subset=['Surface Tension'])

In [7]:
rocky1.head()

,Surface Tension,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,...,F08_C-C_,F08_C-N_,F08_C-O_,F08_O-O_,F09_C-C_,F09_C-O_,F09_O-O_,F10_C-C_,F10_C-O_,F10_O-O_
1,25.90,137.03,9.788,7.659,13.647,9.159,15.918,0.547,0.975,0.654,...,0,0,0,0,0,0,0,0,0,0
2,25.26,123.00,11.182,6.132,10.764,7.398,12.502,0.557,0.978,0.673,...,0,0,0,0,0,0,0,0,0,0
3,24.93,74.14,4.943,7.349,14.745,8.262,17.285,0.490,0.983,0.551,...,0,0,0,0,0,0,0,0,0,0
4,23.18,92.58,6.613,7.461,13.742,8.665,16.020,0.533,0.982,0.619,...,0,0,0,0,0,0,0,0,0,0
5,25.73,120.64,6.032,10.515,19.509,12.188,22.850,0.526,0.975,0.609,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Import and Wrangle Data
#d_X = rocky1.drop(columns=['Density'])
#d_X = rocky1.drop(columns=['Viscosity'])
d_X = rocky1.drop(columns=['Surface Tension'])
#d_y = rocky1['Density']
#d_y = rocky1['Viscosity']
d_y = rocky1['Surface Tension']


In [9]:
train_X, test_X, train_y, test_y = train_test_split(d_X, d_y, test_size = 0.25, random_state = 16)

In [10]:
scaler = StandardScaler() 
train_Xsc = scaler.fit_transform(train_X)
#test_Xsc = scaler.transform(test_X)

In [11]:
#Partial Least Squares
# pls = PLSRegression(n_components=5)  
# X_train_pls = pls.fit_transform(train_Xsc, train_y)[0]
# X_test_pls = pls.transform(test_Xsc)

In [12]:
#Principle Component Analysis
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(train_Xsc)
#X_test_pca = pca.transform(test_Xsc)


In [ ]:
explained_var_ratio = pca.explained_variance_ratio_
cumulative_var_ratio = np.cumsum(explained_var_ratio)

plt.plot(range(1, len(cumulative_var_ratio) + 1), cumulative_var_ratio, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance Ratio vs. Number of Principal Components')
plt.show()

## Elastic Net

In [ ]:
enet = ElasticNet(random_state=22)

In [ ]:
enet_hyperparam = {
    'alpha': np.concatenate([np.logspace(-4, 1, 10)]),
    'l1_ratio': np.concatenate([np.linspace(0, 1, 11)]),
    'max_iter': [1000, 2000],  # Including 1000 and a value close to it for variety
    'selection': ['cyclic', 'random'],  # Including both 'cyclic' and 'random'
    'tol': [0.001, 0.0001, 0.01] 
}
print(enet_hyperparam)

In [ ]:
enet_model_cv = GridSearchCV(estimator=enet,
                       param_grid=enet_hyperparam,
                       scoring='r2',
                       verbose=1,
                       n_jobs=-1,
                       cv=3)

In [ ]:
# enet_model_cv = BayesSearchCV(enet, 
#                               enet_hyperparam,
#                               n_iter=100,
#                               cv=3
# )

In [ ]:
# enet_model_cv = GASearchCV(estimator = enet, 
#                                 cv=3,
#                                 scoring= 'r2',  
#                                 population_size=20,
#                                 generations=35,
#                                 param_grid = enet_hyperparam,
#                                 verbose = False,
#                                 n_jobs = -1,
#                                 keep_top_k=4)

In [ ]:
start = time.time()            # Start Time

enet_model_cv.fit(train_Xsc, train_y)
#enet_model_cv.fit(X_train_pls, train_y)
#enet_model_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', enet_model_cv.best_score_)
print('Initial parameters: ', enet_model_cv.best_params_)

In [ ]:
best_enet = enet_model_cv.best_estimator_
density_pickle = open('density_EN_Full.pickle', 'wb') 
pickle.dump(best_enet, density_pickle) 
density_pickle.close() 

### Elastic Net Predictions

In [ ]:
# Predict Test Set
dy_pred_en = enet_model_cv.predict(test_Xsc)
#dy_pred_en = enet_model_cv.predict(X_test_pls)
#dy_pred_en = enet_model_cv.predict(X_test_pca)

# evaluate the model on test set
r2_density_en = sklearn.metrics.r2_score(test_y, dy_pred_en)
print('R-squared on Test Set: %0.2f' %r2_density_en)

RMSE_test_density_en = sklearn.metrics.mean_squared_error(test_y, dy_pred_en, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_en)

In [ ]:
en_residuals = test_y - dy_pred_en

# Create a DataFrame for the residuals
residuals_df = pd.DataFrame({'Residuals': en_residuals})

# Find the index of the highest residual
max_residual_index = residuals_df['Residuals'].idxmax()

# Drop the row with the highest residual
residuals_df = residuals_df.drop(index=max_residual_index)

residuals_df.hist(bins=25, figsize=(12,8), color="maroon")

plt.xlabel('Residuals')
plt.ylabel('Number of Test Datapoints')
plt.title('Distribution of Residuals: Elastic Net')
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance
feature_names = pd.DataFrame(d_X).columns
perm_importance = permutation_importance(best_enet, test_Xsc, test_y, n_repeats=30, random_state=42)

# Extract importance scores and their standard deviations
importances_mean = perm_importance.importances_mean
importances_std = perm_importance.importances_std

# Get the indices of the top 10 features by importance
top_10_indices = np.argsort(importances_mean)[-10:][::-1]

# Get the names of the top 10 features
top_10_feature_names = [feature_names[i] for i in top_10_indices]

# Print the top 10 features with their importance and confidence intervals
print("Top 10 feature importances with confidence intervals:")
for i in top_10_indices:
    if importances_mean[i] - 2 * importances_std[i] > 0:
        print(f"{feature_names[i]:<8} {importances_mean[i]:.3f} +/- {importances_std[i]:.3f}")

# Plot the top 10 feature importances
top_features = [feature_names[i] for i in top_10_indices]
top_importances = importances_mean[top_10_indices]

plt.figure(figsize=(10, 6))
plt.barh(top_features, top_importances)
plt.xlabel('Permutation Importance')
plt.ylabel('Features')
plt.title('Top 10 Feature Importances for Density Prediction')
plt.show()

# Print the column names of the top 10 features
print("Column names of the top 10 features:", top_10_feature_names)

### Refit

In [ ]:
en_non_zero_mask = enet_model_cv.best_estimator_.coef_ != 0
en_train_Xsc = train_Xsc[:, en_non_zero_mask]
en_test_Xsc = test_Xsc[:, en_non_zero_mask]

In [ ]:
start = time.time()            # Start Time
enet_model_cv.fit(en_train_Xsc, train_y)
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', enet_model_cv.best_score_)
print('Initial parameters: ', enet_model_cv.best_params_)

In [ ]:
dy_pred_en_refit = enet_model_cv.predict(en_test_Xsc)

# evaluate the model on test set
r2_density_en = sklearn.metrics.r2_score(test_y, dy_pred_en_refit)
print('R-squared on Test Set: %0.2f' %r2_density_en)

RMSE_test_density_en = sklearn.metrics.mean_squared_error(test_y, dy_pred_en_refit, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_en)

In [ ]:
print(en_train_Xsc.shape)
print(train_Xsc.shape)

## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso 
lasso_reg = Lasso(random_state=41)

# lasso_hyperparam = {
#     'alpha': np.arange(0.00, 1.1, 0.01)
# } 

lasso_hyperparam = {
    'alpha': np.logspace(-4, 2, 10),  # explore a range from 1e-4 to 1e2
    'max_iter': [1000, 5000, 10000],
    'tol': [1e-4, 1e-3, 1e-2],
    'fit_intercept': [True, False],
    'selection': ['cyclic', 'random']
}

In [ ]:
lassoCV = GridSearchCV(estimator=lasso_reg,
                       param_grid=lasso_hyperparam,
                       scoring='r2',
                       cv=3,
                       verbose=1,
                       n_jobs=-1)

In [ ]:
# lassoCV = BayesSearchCV(
#     Lasso(), 
#     lasso_hyperparam,
#     n_iter=32,
#     cv=5
# )

In [ ]:
# lassoCV = GASearchCV(estimator = lasso_reg, 
#                                 cv=5,
#                                 scoring= 'r2',  
#                                 population_size=20,
#                                 generations=35,
#                                 param_grid = lasso_hyperparam,
#                                 verbose = True,
#                                 n_jobs = -1,
#                                 keep_top_k=4)

In [ ]:
start = time.time()            # Start Time

#lassoCV.fit(train_Xsc, train_y)
#lassoCV.fit(X_train_pls, train_y)
lassoCV.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('R2 score: ', lassoCV.best_score_)
print('Best parameters: ', lassoCV.best_params_)
best_lasso = lassoCV.best_estimator_

In [ ]:
#dy_pred_lasso = lassoCV.predict(test_Xsc)
#dy_pred_lasso = lassoCV.predict(X_test_pls)
dy_pred_lasso = lassoCV.predict(X_test_pca)

# evaluate the model on test set
r2_density_lasso = sklearn.metrics.r2_score(test_y, dy_pred_lasso)
print('R-squared on Test Set: %0.2f' %r2_density_lasso)

RMSE_test_density_lasso = sklearn.metrics.mean_squared_error(test_y, dy_pred_lasso, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_lasso)

In [ ]:
coefficients = best_lasso.coef_

# Get indices of nonzero coefficients
nonzero_indices = np.nonzero(coefficients)[0]

# Print nonzero coefficients
print("Nonzero coefficients:")
for index in nonzero_indices:
    coef_value = coefficients[index]
    # Check if feature names are available
    if hasattr(best_lasso, 'feature_names_in_'):
        feature_name = best_lasso.feature_names_in_[index]
        print(f"{feature_name}: {coef_value:.4f}")
    else:
        print(f"Feature {index}: {coef_value:.4f}")

In [ ]:
# error plot distribution of delta(y_test, y_pred)
lasso_residuals = test_y - dy_pred_lasso

pd.DataFrame({'Residuals': lasso_residuals}).hist(bins=25, figsize=(5,5), color="maroon")

plt.xlabel('Residuals')
plt.ylabel('Number of Test Datapoints')
plt.title('Distribution of Residuals: Lasso Regression');

### Refit

In [ ]:
lasso_non_zero_mask = lassoCV.best_estimator_.coef_ != 0
lassor_train_Xsc = train_Xsc[:, lasso_non_zero_mask]
lassor_test_Xsc = test_Xsc[:, lasso_non_zero_mask]

In [ ]:
start = time.time()            # Start Time
lassoCV.fit(lassor_train_Xsc, train_y)
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', lassoCV.best_score_)
print('Initial parameters: ', lassoCV.best_params_)

In [ ]:
dy_pred_lasso = lassoCV.predict(lassor_test_Xsc)

# evaluate the model on test set
r2_density_lasso = sklearn.metrics.r2_score(test_y, dy_pred_lasso)
print('R-squared on Test Set: %0.2f' %r2_density_lasso)

RMSE_test_density_lasso = sklearn.metrics.mean_squared_error(test_y, dy_pred_lasso, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_lasso)

In [ ]:
print(lassor_train_Xsc.shape)
print(train_Xsc.shape)

## XG Boost

In [ ]:
xg = XGBRegressor()
xg_params = {
    'eta': [0.01, 0.1, 0.3],
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 0.7, 1.0],
}

In [ ]:
xg_random_cv = RandomizedSearchCV(estimator=xg,
                                        param_distributions=xg_params,
                                        n_iter =100,
                                        scoring = 'r2',
                                        cv = 3,
                                        verbose =1,
                                        n_jobs =-1)

In [ ]:
start = time.time()            # Start Time

#xg_random_cv.fit(train_Xsc, train_y)
#xg_random_cv.fit(X_train_pls, train_y)
xg_random_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', xg_random_cv.best_score_)
print('Initial parameters: ', xg_random_cv.best_params_)

In [ ]:
xg_grid_params = {
    'eta': [0.001,0.01,0.1],
    'n_estimators': [100,200,300],
    'max_depth': [9,10,11],
    'subsample': [0.5,0.6,0.7],
}

In [ ]:
xg_cv = GridSearchCV(estimator = xg, 
                    param_grid=xg_grid_params,
                    scoring = 'r2', 
                    cv = 3, 
                    verbose = 1,
                    n_jobs = -1) 

In [ ]:
# xg_cv = BayesSearchCV(
#     xg, 
#     xg_params,
#     n_iter=100,
#     cv=3
# )

In [ ]:
# xg_cv = GASearchCV(estimator = xg, 
#                                 cv=3,
#                                 scoring= 'r2',  
#                                 population_size=20,
#                                 generations=35,
#                                 param_grid = xg_params,
#                                 verbose = True,
#                                 n_jobs = -1,
#                                 keep_top_k=4)

In [ ]:
start = time.time()            # Start Time

#xg_cv.fit(train_Xsc, train_y)
#xg_cv.fit(X_train_pls, train_y)
xg_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Improved score: ', xg_cv.best_score_)
print('Improved parameters: ', xg_cv.best_params_)

In [ ]:
best_xg_reg = xg_cv.best_estimator_
feature_names = pd.DataFrame(train_Xsc).columns

In [ ]:
xg_importance = best_xg_reg.feature_importances_

# Storing feature importance as a dataframe
xg_feature_imp = pd.DataFrame(list(zip(feature_names, xg_importance)),
               columns = ['Feature', 'Importance'])

xg_feature_imp = xg_feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
xg_feature_imp_nonzero = xg_feature_imp[xg_feature_imp['Importance'] > 0.05]
xg_fig = plt.figure(figsize=(10, 5))
plt.barh(xg_feature_imp_nonzero['Feature'], xg_feature_imp_nonzero['Importance'], color = ['blue', 'green'])

plt.xlabel("Importance", fontsize = 12)
plt.ylabel("Input Feature", fontsize = 10)
plt.title('Which features are the most important for density prediction?', fontsize = 12) 
plt.yticks(fontsize = 8) # fontsize of yticks
plt.xticks(fontsize = 8) # fontsize of xticks

#plt.tight_layout();
#xg_fig.savefig('xg_aqueous_feature_imp.svg')

In [ ]:
#dy_pred_xg = xg_cv.predict(test_Xsc)
#dy_pred_xg = xg_cv.predict(X_test_pls)
dy_pred_xg = xg_cv.predict(X_test_pca)

# evaluate the model on test set
r2_density_xg = sklearn.metrics.r2_score(test_y, dy_pred_xg)
print('R-squared on Test Set: %0.2f' %r2_density_xg)

RMSE_test_density_xg = sklearn.metrics.mean_squared_error(test_y, dy_pred_xg, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_xg)

### Refit

In [ ]:
xg_non_zero_mask = xg_importance != 0
xg_train_Xsc = train_Xsc[:, xg_non_zero_mask]
xg_test_Xsc = test_Xsc[:, xg_non_zero_mask]

In [ ]:
start = time.time()            # Start Time
xg_cv.fit(xg_train_Xsc, train_y)
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', xg_cv.best_score_)
print('Initial parameters: ', xg_cv.best_params_)

In [ ]:
dy_pred_xg = xg_cv.predict(xg_test_Xsc)

# evaluate the model on test set
r2_density_xg = sklearn.metrics.r2_score(test_y, dy_pred_xg)
print('R-squared on Test Set: %0.2f' %r2_density_xg)

RMSE_test_density_xg = sklearn.metrics.mean_squared_error(test_y, dy_pred_xg, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_xg)

In [ ]:
print(xg_train_Xsc.shape)
print(train_Xsc.shape)

## Random Forest

In [ ]:
rf = RandomForestRegressor(random_state=68)

In [ ]:
rf_params = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],}
rf_params

In [ ]:
rf_random_model_cv = RandomizedSearchCV(estimator=rf,
                                        param_distributions=rf_params,
                                        n_iter =100,
                                        scoring = 'r2',
                                        cv = 3,
                                        verbose =1,
                                        n_jobs =-1)

In [ ]:
start = time.time()            # Start Time

#rf_random_model_cv.fit(train_Xsc, train_y)
#rf_random_model_cv.fit(X_train_pls, train_y)
rf_random_model_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Initial score: ', rf_random_model_cv.best_score_)
print('Initial parameters: ', rf_random_model_cv.best_params_)

In [ ]:
rf_grid_params = {
    'n_estimators': [400,500,1000],
    'max_depth': [8,10,12],
    'min_samples_split': [8,10,12],
    'min_samples_leaf': [4,5,6]
}

In [ ]:
rf_grid_cv = GridSearchCV(estimator = rf, 
                                param_grid = rf_grid_params, 
                                scoring= 'r2', 
                                cv = 3, 
                                verbose = 1,
                                n_jobs = -1)

In [ ]:
# rf_grid_cv = BayesSearchCV(
#     rf, rf_params,
#     n_iter=200,
#     cv=3
# )

In [ ]:
# rf_grid_cv = GASearchCV(estimator = rf, 
#                                 cv=3,
#                                 scoring= 'r2',  
#                                 population_size=20,
#                                 generations=35,
#                                 param_grid = rf_params,
#                                 verbose = True,
#                                 n_jobs = -1,
#                                 keep_top_k=4)

In [ ]:
start = time.time()            # Start Time

#rf_grid_cv.fit(train_Xsc, train_y)  
#rf_grid_cv.fit(X_train_pls, train_y)
rf_grid_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Improved score: ', rf_grid_cv.best_score_)
print('Improved parameters: ', rf_grid_cv.best_params_)

In [ ]:
best_rf_reg = rf_grid_cv.best_estimator_
feature_names = pd.DataFrame(train_Xsc).columns

In [ ]:
rf_importance = best_rf_reg.feature_importances_

# Storing feature importance as a dataframe
rf_feature_imp = pd.DataFrame(list(zip(feature_names, rf_importance)),
               columns = ['Feature', 'Importance'])

# rf_feature_imp = pd.DataFrame(list(zip(X_train_pca.columns, rf_importance)),
#                columns = ['Feature', 'Importance'])

rf_feature_imp = rf_feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)


# Bar plot
rf_feature_imp_nonzero = rf_feature_imp[rf_feature_imp['Importance'] > 0.01]
rf_fig = plt.figure(figsize=(10, 5))
plt.barh(rf_feature_imp['Feature'], rf_feature_imp['Importance'], color = ['blue', 'green'])

plt.xlabel("Importance", fontsize = 12)
plt.ylabel("Input Feature", fontsize = 12)
plt.title('Which features are the most important for density prediction?', fontsize = 12) 
plt.yticks(fontsize = 8) # fontsize of yticks
plt.xticks(fontsize = 8) # fontsize of xticks

plt.tight_layout();
rf_fig.savefig('rf_aqueous_feature_imp.svg')

In [ ]:
#dy_pred_rf = rf_grid_cv.predict(test_Xsc)
#dy_pred_rf = rf_grid_cv.predict(X_test_pls)
dy_pred_rf = rf_grid_cv.predict(X_test_pca)

# evaluate the model on test set
r2_density_rf = sklearn.metrics.r2_score(test_y, dy_pred_rf)
print('R-squared on Test Set: %0.2f' %r2_density_rf)

RMSE_test_density_rf = sklearn.metrics.mean_squared_error(test_y, dy_pred_rf, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_rf)

### Refit

In [ ]:
rf_non_zero_mask = rf_importance != 0
rf_train_Xsc = train_Xsc[:, rf_non_zero_mask]
rf_test_Xsc = test_Xsc[:, rf_non_zero_mask]

In [ ]:
start = time.time()            # Start Time
rf_random_model_cv.fit(rf_train_Xsc, train_y)  
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Improved score: ', rf_grid_cv.best_score_)
print('Improved parameters: ', rf_grid_cv.best_params_)

In [ ]:
dy_pred_rf = rf_random_model_cv.predict(rf_test_Xsc)

# evaluate the model on test set
r2_density_rf = sklearn.metrics.r2_score(test_y, dy_pred_rf)
print('R-squared on Test Set: %0.2f' %r2_density_rf)

RMSE_test_density_rf = sklearn.metrics.mean_squared_error(test_y, dy_pred_rf, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_rf)

## Support Vector Regression

In [13]:
svr = SVR()
kernels = ['linear', 'poly', 'rbf']

svr_params = {
    'kernel' : kernels,
    'C': np.logspace(-3, 3, num=7), 
    'gamma': np.logspace(-3, 2, num=6)
    }
# svr_params = {
#     'kernel' : Categorical(['linear', 'poly', 'rbf']),
#     'C': Continuous(1e-3, 1e3, distribution='log-uniform'),  # Continuous values from 1e-3 to 1e3
#     'gamma': Continuous(1e-3, 1e2, distribution='log-uniform') 
#     }

In [ ]:
# svr_cv = GridSearchCV(estimator = svr, 
#                                 param_grid = svr_params, 
#                                 scoring= 'r2',
#                                 cv=3,  
#                                 verbose = 1,
#                                 n_jobs = -1)

In [14]:
svr_cv = BayesSearchCV(
    svr, 
    svr_params,
    n_iter=100,
    cv=3
)            

In [ ]:
# svr_cv = GASearchCV(estimator = svr, 
#                                 cv=3,
#                                 scoring= 'r2',  
#                                 population_size=20,
#                                 generations=35,
#                                 param_grid = svr_params,
#                                 verbose = False,
#                                 n_jobs = -1,
#                                 keep_top_k=4)



In [15]:
start = time.time()            # Start Time

#svr_cv.fit(train_Xsc, train_y)
#svr_cv.fit(X_train_pls, train_y)
svr_cv.fit(X_train_pca, train_y)

stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

/Users/cameronlutz/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, 100.0, 'linear'] before, using random point [0.001, 100.0, 'poly']
  warnings.warn(
/Users/cameronlutz/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, 100.0, 'linear'] before, using random point [0.1, 0.1, 'linear']
  warnings.warn(
/Users/cameronlutz/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, 0.01, 'linear'] before, using random point [0.001, 10.0, 'poly']
  warnings.warn(
/Users/cameronlutz/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [0.01, 0.01, 'linear'] before, using random point [1.0, 0.001, 'poly']
  warnings.warn(
/Users/cameronlutz/anaconda3/lib/python3.11/site-packag

Training time: 1062.9075329303741s


In [16]:
# print best parameter after tuning 
print('Improved score: ', svr_cv.best_score_)
print(svr_cv.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(svr_cv.best_estimator_) 

best_svr = svr_cv.best_estimator_

Improved score:  0.3247468676391685
OrderedDict([('C', 10.0), ('gamma', 0.001), ('kernel', 'rbf')])
SVR(C=10.0, gamma=0.001)


In [ ]:
#dy_pred_svr = svr_cv.predict(test_Xsc)
#dy_pred_svr = svr_cv.predict(X_test_pls)
dy_pred_svr = svr_cv.predict(X_test_pca)

# evaluate the model on test set
r2_density_svr = sklearn.metrics.r2_score(test_y, dy_pred_svr)
print('R-squared on Test Set: %0.2f' %r2_density_svr)

RMSE_test_density_svr = sklearn.metrics.mean_squared_error(test_y, dy_pred_svr, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_svr)

In [ ]:
# viscosity_pickle = open('visc_SVR_Full.pickle', 'wb') 
# pickle.dump(best_svr, viscosity_pickle) 
# viscosity_pickle.close() 

In [17]:
surface_pickle = open('surface_SVR_PCA.pickle', 'wb') 
pickle.dump(best_svr, surface_pickle) 
surface_pickle.close() 

In [ ]:
svr_residuals = test_y - dy_pred_svr

pd.DataFrame({'Residuals': svr_residuals}).hist(bins=25, figsize=(12,8), color="maroon")

plt.xlabel('Residuals')
plt.ylabel('Number of Test Datapoints')
plt.title('Distribution of SVR Residuals: Surface Tension');

### Refit

In [ ]:
if svr_cv.best_params_['kernel'] == 'linear':
    svr_non_zero_mask = best_svr.coef_ != 0
    svr_non_zero_mask = svr_non_zero_mask.ravel()  # Ensure it's a 1D array

    svr_train_Xsc = train_Xsc[:, svr_non_zero_mask]
    svr_test_Xsc = test_Xsc[:, svr_non_zero_mask]

    # Refit the model using only the selected features
    best_svr.fit(svr_train_Xsc, train_y)
    dy_pred_svr_refit = best_svr.predict(svr_test_Xsc)
    r2_density_svr_refit = r2_score(test_y, dy_pred_svr_refit)
    RMSE_test_density_svr_refit = mean_squared_error(test_y, dy_pred_svr_refit, squared=False)

    print('R-squared on Test Set after refit: %0.2f' % r2_density_svr_refit)
    print('RMSE on Test Set after refit: %0.2f' % RMSE_test_density_svr_refit)

In [ ]:
svr_non_zero_mask = svr_cv.best_estimator_.coef_ != 0
svr_train_Xsc = train_Xsc[:, svr_non_zero_mask]
svr_test_Xsc = test_Xsc[:, svr_non_zero_mask]

In [ ]:
start = time.time()            # Start Time
svr_cv.fit(svr_train_Xsc, train_y)
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

In [ ]:
print('Improved score: ', svr_cv.best_score_)
print('Improved parameters: ', svr_cv.best_params_)

In [ ]:
dy_pred_svr = svr_cv.predict(svr_test_Xsc)

# evaluate the model on test set
r2_density_svr = sklearn.metrics.r2_score(test_y, dy_pred_svr)
print('R-squared on Test Set: %0.2f' %r2_density_svr)

RMSE_test_density_svr = sklearn.metrics.mean_squared_error(test_y, dy_pred_svr, squared=False)
print('RMSE on Test Set: %0.2f' %RMSE_test_density_svr)

## MAPIE Module

In [ ]:
# Define mapie regressor
mapie = MapieRegressor(estimator = best_svr, # Prediction Model to use
                       n_jobs = -1,
                       agg_function = "median",
                       random_state = 42)

# Fit mapie regressor on training data
mapie.fit(train_Xsc, train_y)

alpha = 0.1 # for 90% target coverage

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_Xsc, alpha = alpha)

In [ ]:
# Predicted values
y_test_pred

In [ ]:
# Prediction Intervals
y_test_pis

In [ ]:
# Storing results in a dataframe
predictions = test_y.to_frame()
predictions.columns = ['Actual Value']
predictions["Predicted Value"] = y_test_pred.round(2)
predictions["Lower Value"] = y_test_pis.reshape(-1,2)[:,0].round(2)
predictions["Upper Value"] = y_test_pis.reshape(-1,2)[:,1].round(2)

# Take a quick look
predictions

In [ ]:
predictions["Error"] = predictions["Predicted Value"] - predictions["Actual Value"]

predictions["Error_upper"] =   (predictions["Upper Value"] - predictions["Predicted Value"])
predictions["Error_lower"] =  -(predictions["Predicted Value"] - predictions["Lower Value"])

# Sort by total interval width
predictions["Interval_width"] = predictions["Upper Value"] - predictions["Lower Value"]
sorted_predictions = predictions.sort_values(by=['Interval_width']).reset_index(drop=True)
sorted_predictions = sorted_predictions.drop(sorted_predictions.index[-1])
sorted_predictions

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

plt.plot(sorted_predictions["Error"], 'o', markersize = 3, label = "Error (y_pred - y_true)")

plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Error_lower"],
                 sorted_predictions["Error_upper"],
                 alpha=0.5, color="grey", label = "Prediction Interval")

ax.axline([0, 0], [1, 0], color = "red", linestyle='--', lw=2, zorder=3, label="y_true")
plt.xticks([])
plt.xlim([0, len(sorted_predictions)])
plt.ylabel("Errors")
plt.legend(loc="upper left", fontsize=14)
plt.show()

In [ ]:
# count number of points outside of predicted interval
sorted_predictions["is_outside_range"] = 0
sorted_predictions["is_outside_range"] = sorted_predictions["is_outside_range"].where((
    (sorted_predictions["Error"] < sorted_predictions["Error_upper"]) & (sorted_predictions["Error"] > sorted_predictions["Error_lower"]) ),
    other=1)

print(round(100-(100/len(sorted_predictions))*sorted_predictions["is_outside_range"].sum(),1))

In [ ]:
# count number of prediction intervals that actually contain the ground truth value
sorted_predictions["gt_within_PI"] = 0
sorted_predictions["gt_within_PI"] = sorted_predictions["gt_within_PI"].where((
    (sorted_predictions["Actual Value"] < sorted_predictions["Upper Value"]) & (sorted_predictions["Actual Value"] > sorted_predictions["Lower Value"]) ),
    other=1)

print(round(100-(100/len(sorted_predictions))*sorted_predictions["gt_within_PI"].sum(),1))

In [ ]:
# re-sort for plot
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

fig, ax = plt.subplots(figsize=(15, 9))

plt.plot(sorted_predictions["Actual Value"], 'o', markersize=3, label="Actual Value")

plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.5, color="grey", label="Prediction Interval")

plt.xticks([],fontsize=14)
plt.xlim([0, len(sorted_predictions)])
plt.ylabel("True value", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.show()

## SHAP Module

In [ ]:
import shap
explainer = shap.Explainer(best_svr.predict, X_train_pca, feature_names=feature_names)

# Calculate the minimum required evaluations
num_features = X_train_pca.shape[1]
min_evals = 2 * num_features + 1

# Calculate SHAP values for the test set with increased max_evals
shap_values = explainer(X_test_pca, max_evals=min_evals)

# Plot SHAP values for the first prediction
shap.summary_plot(shap_values, X_test_pca, feature_names=feature_names)

In [ ]:
shap.plots.waterfall(shap_values[0], max_display=11)

In [ ]:
#Global Model Interpretation
shap.plots.beeswarm(shap_values)

## Feature Importance for SVR


In [ ]:
feature_names = pd.DataFrame(d_X).columns

In [ ]:
from sklearn.inspection import permutation_importance
#perm_importance = permutation_importance(svr_cv, test_Xsc, test_y, n_repeats=30, random_state=42)
perm_importance = permutation_importance(svr_cv, X_test_pca, test_y, n_repeats=30, random_state=42)

# Extract importance scores and their standard deviations
importances_mean = perm_importance.importances_mean
importances_std = perm_importance.importances_std

# Get the indices of the top 10 features by importance
top_10_indices = np.argsort(importances_mean)[-10:][::-1]

# Get the names of the top 10 features
top_10_feature_names = [feature_names[i] for i in top_10_indices]

# Print the top 10 features with their importance and confidence intervals
print("Top 10 feature importances with confidence intervals:")
for i in top_10_indices:
    if importances_mean[i] - 2 * importances_std[i] > 0:
        print(f"{feature_names[i]:<8} {importances_mean[i]:.3f} +/- {importances_std[i]:.3f}")

# Plot the top 10 feature importances
top_features = [feature_names[i] for i in top_10_indices]
top_importances = importances_mean[top_10_indices]

plt.figure(figsize=(10, 6))
plt.barh(top_features, top_importances)
plt.xlabel('Permutation Importance')
plt.ylabel('Features')
plt.title('Top 10 Feature Importances for Surface Tension Prediction')
plt.show()

# Print the column names of the top 10 features
print("Column names of the top 10 features:", top_10_feature_names)

# PREDICTION

In [13]:
maywheather = pd.read_excel('FINAL-predictionset.xlsx')
maywheather.head()

,ID,Molecule_Name,SMILES,MW,AMW,Sv,Se,Sp,Si,Mv,...,F08_C-C_,F08_C-N_,F08_C-O_,F08_O-O_,F09_C-C_,F09_C-O_,F09_O-O_,F10_C-C_,F10_C-O_,F10_O-O_
0,273,Dipropylene glycol diacrylate,CC(COCC(C)OC(=O)C=C)OC(=O)C=C,242.30,6.923,20.315,35.589,21.125,39.784,0.580,...,5,0,4,2,4,4,0,3,2,1
1,274,Ethoxylated 3 EO trimethylolpropane triacrylate,CCC(COCCOC(=O)C=C)(COCCOC(=O)C=C)COCCOC(=O)C=C,428.53,6.912,35.862,63.083,37.273,70.528,0.578,...,18,0,21,0,18,15,6,15,12,3
2,275,Ethoxylated 5 EO pentaerythritol tetraacrylate,C=CC(=O)OCC(COC(=O)C=C)(COC(=O)C=C)COC(=O)C=C,352.37,7.830,27.986,46.454,28.250,50.827,0.622,...,18,0,12,6,12,12,0,6,0,0
3,276,Dipentaerythritol hexaacrylate,C=CC(=O)OCC(COCC(COC(=O)C=C)(COC(=O)C=C)COC(=O...,578.62,7.715,46.248,77.276,46.852,84.781,0.617,...,48,0,18,15,36,48,0,33,18,18
4,277,Octyl Acrylate,CCCCCCCCOC(=O)C=C,184.31,5.585,17.698,32.491,19.523,37.571,0.536,...,3,0,2,0,3,1,0,2,1,0


In [14]:
maywheather = maywheather.drop(columns=['ID','Molecule_Name','SMILES'])
test_X = maywheather
test_X.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,F08_C-C_,F08_C-N_,F08_C-O_,F08_O-O_,F09_C-C_,F09_C-O_,F09_O-O_,F10_C-C_,F10_C-O_,F10_O-O_
0,242.30,6.923,20.315,35.589,21.125,39.784,0.580,1.017,0.604,1.137,...,5,0,4,2,4,4,0,3,2,1
1,428.53,6.912,35.862,63.083,37.273,70.528,0.578,1.017,0.601,1.138,...,18,0,21,0,18,15,6,15,12,3
2,352.37,7.830,27.986,46.454,28.250,50.827,0.622,1.032,0.628,1.129,...,18,0,12,6,12,12,0,6,0,0
3,578.62,7.715,46.248,77.276,46.852,84.781,0.617,1.030,0.625,1.130,...,48,0,18,15,36,48,0,33,18,18
4,184.31,5.585,17.698,32.491,19.523,37.571,0.536,0.985,0.592,1.139,...,3,0,2,0,3,1,0,2,1,0


In [15]:
qtest_Xsc = scaler.transform(test_X)

In [16]:
# Retain 95% of the variance
test_Xsc = pca.transform(qtest_Xsc)

In [ ]:
with open('density_EN_Full.pickle', 'rb') as density_pickle_pred:
    density_prediction_model = pickle.load(density_pickle_pred)

# Make predictions using the model
d_predictions = density_prediction_model.predict(test_Xsc)

# Create a DataFrame with the predictions
d_predictions_df = pd.DataFrame(d_predictions, columns=['Predicted Density'])

# Optionally, combine with the original test data for reference
d_final_df = pd.concat([test_X.reset_index(drop=True), d_predictions_df], axis=1)

# Display the final DataFrame
print(d_final_df.head())

In [ ]:
with open('visc_SVR_Full.pickle', 'rb') as viscosity_pickle_pred:
    viscosity_prediction_model = pickle.load(viscosity_pickle_pred)

# Make predictions using the model
v_predictions = viscosity_prediction_model.predict(test_Xsc)

# Create a DataFrame with the predictions
v_predictions_df = pd.DataFrame(v_predictions, columns=['Predicted Viscosity'])

# Optionally, combine with the original test data for reference
v_final_df = pd.concat([test_X.reset_index(drop=True), v_predictions_df], axis=1)

# Display the final DataFrame
print(v_final_df.head())

In [17]:
with open('surface_SVR_PCA.pickle', 'rb') as surface_pickle_pred:
    surface_prediction_model = pickle.load(surface_pickle_pred)

# Make predictions using the model
s_predictions = surface_prediction_model.predict(test_Xsc)

# Create a DataFrame with the predictions
s_predictions_df = pd.DataFrame(s_predictions, columns=['Predicted Surface Tension'])

# Optionally, combine with the original test data for reference
s_final_df = pd.concat([test_X.reset_index(drop=True), s_predictions_df], axis=1)

# Display the final DataFrame
print(s_final_df.head())

       MW    AMW      Sv      Se      Sp      Si     Mv     Me     Mp     Mi  \
0  242.30  6.923  20.315  35.589  21.125  39.784  0.580  1.017  0.604  1.137   
1  428.53  6.912  35.862  63.083  37.273  70.528  0.578  1.017  0.601  1.138   
2  352.37  7.830  27.986  46.454  28.250  50.827  0.622  1.032  0.628  1.129   
3  578.62  7.715  46.248  77.276  46.852  84.781  0.617  1.030  0.625  1.130   
4  184.31  5.585  17.698  32.491  19.523  37.571  0.536  0.985  0.592  1.139   

   ...  F08_C-N_  F08_C-O_  F08_O-O_  F09_C-C_  F09_C-O_  F09_O-O_  F10_C-C_  \
0  ...         0         4         2         4         4         0         3   
1  ...         0        21         0        18        15         6        15   
2  ...         0        12         6        12        12         0         6   
3  ...         0        18        15        36        48         0        33   
4  ...         0         2         0         3         1         0         2   

   F10_C-O_  F10_O-O_  Predicted Surfa

In [18]:
from mapie.regression import MapieRegressor
# density_prediction_model
# viscosity_prediction_model
# surface_prediction_model

# Define mapie regressor
#mapie = MapieRegressor(estimator = density_prediction_model, # Prediction Model to use
#mapie = MapieRegressor(estimator = viscosity_prediction_model,
mapie = MapieRegressor(estimator = surface_prediction_model,                    
                       n_jobs = -1,
                       agg_function = "median",
                       random_state = 42)

# Fit mapie regressor on training data
#mapie.fit(test_Xsc, d_predictions) 
#mapie.fit(test_Xsc, v_predictions) 
mapie.fit(test_Xsc, s_predictions) 

alpha = 0.1 # for 90% target coverage

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_Xsc, alpha = alpha)

In [19]:
# Predicted values
y_test_pred

array([36.61595683, 37.2079704 , 37.21410981, 37.21465132, 29.03196736,
       28.95221304, 30.1844316 , 30.18428312, 30.80457717, 31.33559595,
       31.8618215 , 32.37245862, 33.3089074 , 32.9872712 , 35.65426963,
       30.09684001, 32.4529778 , 33.37986564, 36.39867829, 36.56979462,
       32.90217513, 35.62169529, 36.42649853, 31.38874103, 32.7767699 ,
       36.12647787, 37.30373607, 32.21939771, 32.19700902, 30.49859751,
       35.73352336, 37.03438834, 37.37428558, 37.16879187, 35.45820566,
       36.8809405 , 37.04145454, 36.99642429, 36.80200731, 40.45710502,
       35.56116796, 35.21215384, 31.2368502 , 45.1854161 , 38.6308184 ])

In [20]:
# Prediction Intervals
y_test_pis

array([[[33.69580728],
        [39.53712235]],

       [[34.23526772],
        [40.12905134]],

       [[34.24210586],
        [40.13535916]],

       [[34.29346016],
        [40.13477523]],

       [[26.14835588],
        [31.95221641]],

       [[26.03130056],
        [31.87261563]],

       [[27.26371503],
        [33.10503011]],

       [[27.26202016],
        [33.07709758]],

       [[27.88376473],
        [33.68760463]],

       [[28.41479218],
        [34.25610725]],

       [[28.94179694],
        [34.78311201]],

       [[29.45209106],
        [35.29340613]],

       [[30.38522257],
        [36.24285248]],

       [[30.10427936],
        [35.90777116]],

       [[32.68153431],
        [38.53791068]],

       [[27.17621574],
        [33.07008819]],

       [[29.53214021],
        [35.37345528]],

       [[30.45897923],
        [36.3002943 ]],

       [[33.42586803],
        [39.28197863]],

       [[33.64994085],
        [39.49125592]],

       [[29.92946474],
        [35.78516

In [21]:
y_test_pred = y_test_pred.flatten()
lower_bounds = y_test_pis[:, 0].flatten()
upper_bounds = y_test_pis[:, 1].flatten()

In [22]:
# Storing results in a dataframe
predictions = pd.DataFrame({
    'Lower Bound': lower_bounds.round(2),
    'Predicted Value': y_test_pred.round(2),
    'Upper Bound': upper_bounds.round(2)
})
# Take a quick look
predictions
#predictions.to_excel('density-ints.xlsx', index=False)
#predictions.to_excel('visc-ints.xlsx', index=False)
predictions.to_excel('surface-ints.xlsx', index=False)

### MISC MAPIE

In [ ]:
predictions["Error"] = predictions["Predicted Value"] - predictions["Actual Value"]

predictions["Error_upper"] =   (predictions["Upper Value"] - predictions["Predicted Value"])
predictions["Error_lower"] =  -(predictions["Predicted Value"] - predictions["Lower Value"])

# Sort by total interval width
predictions["Interval_width"] = predictions["Upper Value"] - predictions["Lower Value"]
sorted_predictions = predictions.sort_values(by=['Interval_width']).reset_index(drop=True)
sorted_predictions = sorted_predictions.drop(sorted_predictions.index[-1])
sorted_predictions

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

plt.plot(sorted_predictions["Error"], 'o', markersize = 3, label = "Error (y_pred - y_true)")

plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Error_lower"],
                 sorted_predictions["Error_upper"],
                 alpha=0.5, color="grey", label = "Prediction Interval")

ax.axline([0, 0], [1, 0], color = "red", linestyle='--', lw=2, zorder=3, label="y_true")
plt.xticks([])
plt.xlim([0, len(sorted_predictions)])
plt.ylabel("Errors")
plt.legend(loc="upper left", fontsize=14)
plt.show()

In [ ]:
# count number of points outside of predicted interval
sorted_predictions["is_outside_range"] = 0
sorted_predictions["is_outside_range"] = sorted_predictions["is_outside_range"].where((
    (sorted_predictions["Error"] < sorted_predictions["Error_upper"]) & (sorted_predictions["Error"] > sorted_predictions["Error_lower"]) ),
    other=1)

print(round(100-(100/len(sorted_predictions))*sorted_predictions["is_outside_range"].sum(),1))

In [ ]:
# count number of prediction intervals that actually contain the ground truth value
sorted_predictions["gt_within_PI"] = 0
sorted_predictions["gt_within_PI"] = sorted_predictions["gt_within_PI"].where((
    (sorted_predictions["Actual Value"] < sorted_predictions["Upper Value"]) & (sorted_predictions["Actual Value"] > sorted_predictions["Lower Value"]) ),
    other=1)

print(round(100-(100/len(sorted_predictions))*sorted_predictions["gt_within_PI"].sum(),1))

In [ ]:
# re-sort for plot
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

fig, ax = plt.subplots(figsize=(15, 9))

plt.plot(sorted_predictions["Actual Value"], 'o', markersize=3, label="Actual Value")

plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.5, color="grey", label="Prediction Interval")

plt.xticks([],fontsize=14)
plt.xlim([0, len(sorted_predictions)])
plt.ylabel("True value", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.show()